#### main

### main code 

In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import io
import os

print("TensorFlow version:", tf.__version__)

app = Flask(__name__)
CORS(app)  # Enable CORS for all origins


MODEL_PATHS = {
    "model1Name": r'C:\\Users\\USER\\Desktop\\Ready data\\Model_3\\model_resnet50_best_of_best_7_1.h5',
    "model2Name": r"C:\\Users\\USER\\Downloads\\model_resnet50_5_clases.h5",
}


CLASS_LABELS = {
    "model1Name": ["Cheeseweed", "Coriander", "Ecballium", "MilkThistle", "Nettles", "Oleander", "Parsley"],
    "model2Name": ["Cheeseweed", "Ecballium", "MilkThistle", "Nettles", "Oleander"],
}


models = {}
for name, path in MODEL_PATHS.items():
    if os.path.exists(path):
        try:
            models[name] = tf.keras.models.load_model(path)
            print(f"{name} loaded successfully.")
        except Exception as e:
            print(f"Error loading {name}: {e}")
    else:
        print(f"{name} not found at {path}")


def preprocess_image(image_path, target_size=(224, 224)):
   
    img = load_img(image_path, target_size=target_size) 
    img_array = img_to_array(img) 
    img_array = np.expand_dims(img_array, axis=0)  
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)  
    return img_array


def predict_top_classes(image_path, model, class_labels, top_n=3):
   
    preprocessed_image = preprocess_image(image_path)
    predictions = model.predict(preprocessed_image)[0]  # Get the predictions for all classes
    top_indices = np.argsort(predictions)[-top_n:][::-1]  # Get indices of top N classes
    top_predictions = [
        {"label": class_labels[i], "confidence": float(predictions[i])}
        for i in top_indices
    ]
    return top_predictions


@app.route('/upload', methods=['POST'])
def upload():
    if 'image' not in request.files or 'model' not in request.form:
        return jsonify({"error": "Missing file or model parameter"}), 400

    selected_model = request.form['model']
    if selected_model not in models:
        return jsonify({"error": f"Model {selected_model} not available"}), 400

    print(f"Using model: {selected_model}")  

    file = request.files['image']
    try:
        image = Image.open(io.BytesIO(file.read()))
        image_path = "temp_image.jpg"
        image.save(image_path) 
    except Exception as e:
        return jsonify({"error": f"Invalid image format: {e}"}), 400

    
    class_labels = CLASS_LABELS[selected_model]

    try:
        top_predictions = predict_top_classes(image_path, models[selected_model], class_labels, top_n=3)
        print(f"Top Predictions: {top_predictions}")
        os.remove(image_path)  
    except Exception as e:
        return jsonify({"error": f"Prediction failed: {e}"}), 500

    return jsonify({
        "predictions": top_predictions
    })


if __name__ == '__main__':
    app.run(debug=True, host="0.0.0.0", port=5000, use_reloader=False)


TensorFlow version: 2.10.0
model1Name loaded successfully.
model2Name loaded successfully.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.34:5000
Press CTRL+C to quit


Using model: model1Name
1/1 [==============================] - 8s 8s/step


192.168.0.34 - - [06/Jan/2025 13:12:03] "POST /upload HTTP/1.1" 200 -


Top Predictions: [{'label': 'Nettles', 'confidence': 0.6564901471138}, {'label': 'Cheeseweed', 'confidence': 0.31906402111053467}, {'label': 'Ecballium', 'confidence': 0.013545366935431957}]
Using model: model1Name
1/1 [==============================] - 0s 384ms/step


...

Top Predictions: [{'label': 'Oleander', 'confidence': 0.9971056580543518}, {'label': 'MilkThistle', 'confidence': 0.0009314596536569297}, {'label': 'Ecballium', 'confidence': 0.0008893557824194431}]
Using model: model1Name
1/1 [==============================] - 0s 17ms/step


192.168.0.34 - - [06/Jan/2025 13:31:53] "POST /upload HTTP/1.1" 200 -


Top Predictions: [{'label': 'Parsley', 'confidence': 0.7287545800209045}, {'label': 'Ecballium', 'confidence': 0.09541109949350357}, {'label': 'Oleander', 'confidence': 0.05780785158276558}]
Using model: model1Name
1/1 [==============================] - 0s 22ms/step


192.168.0.34 - - [06/Jan/2025 13:34:07] "POST /upload HTTP/1.1" 200 -


Top Predictions: [{'label': 'Parsley', 'confidence': 0.7287545800209045}, {'label': 'Ecballium', 'confidence': 0.09541109949350357}, {'label': 'Oleander', 'confidence': 0.05780785158276558}]


In [ ]:
#test images

In [22]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the trained ResNet50 model
model_path = 'model_resnet50_best_of_best_7_1.h5'  # Replace with the path to your model
model = tf.keras.models.load_model(model_path)

# Function to preprocess input image for ResNet50
def preprocess_image(image_path, target_size=(224, 224)):
    """
    Preprocesses an input image for ResNet50.
    
    Args:
    - image_path: Path to the input image.
    - target_size: Target size for the input image.
    
    Returns:
    - Preprocessed image ready for prediction.
    """
    img = load_img(image_path, target_size=target_size)  # Load image with target size
    img_array = img_to_array(img)  # Convert to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)  # Normalize as per ResNet50 requirements
    return img_array

# Function to make predictions with confidence scores
def predict_image_with_all_confidences(image_path, class_labels):
    """
    Predicts the class of an image using the ResNet50 model and returns the confidence scores for all classes.
    
    Args:
    - image_path: Path to the image to be predicted.
    - class_labels: List of class labels corresponding to the model output.
    
    Returns:
    - Dictionary of class labels and their confidence scores.
    """
    preprocessed_image = preprocess_image(image_path)
    predictions = model.predict(preprocessed_image)[0]  # Get the predictions for all classes
    confidences = {class_labels[i]: float(predictions[i]) for i in range(len(class_labels))}
    return confidences

# Example usage
if __name__ == "__main__":
    # Define the class labels (replace with your actual class labels)
    class_labels = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_p']  # Update with your model's classes

    # Path to the test image
    test_image_path = 'test8.jpg'  # Replace with the path to the image you want to test

    # Predict the confidence scores for all classes
    confidences = predict_image_with_all_confidences(test_image_path, class_labels)
    
    print("Confidence scores for all classes:")
    for label, confidence in confidences.items():
        print(f"{label}: {confidence:.2f}")


1/1 [==============================] - 0s 419ms/step
Confidence scores for all classes:
class_0: 0.00
class_1: 0.00
class_2: 0.00
class_3: 0.00
class_4: 0.00
class_5: 0.97
class_p: 0.01
